In [1]:
import os
# Backend change. Don't run this if you don't know what it is
os.environ['KERAS_BACKEND'] = 'theano'
#os.environ['THEANO_FLAGS'] = 'device=opencl1:0, floatX=float32'


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import pandas as pa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


Using Theano backend.


Dataset Preprocess


In [17]:

#Reading csv file
dataset = pa.read_csv("../Data/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")


In [18]:
#Displaying a subset of data
dataset.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [19]:
dataset.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [20]:
#Dropping the target attribute from the dataset and making it equal to X
#Target attribute created as y
X = dataset.drop(columns=['customerID', 'Churn'])
y    = dataset[['Churn']]

In [21]:

#Need to encode non-numerical data
from sklearn.preprocessing import LabelEncoder
lb_coder = LabelEncoder()
y['Churn'] = lb_coder.fit_transform(y['Churn'])
for i in range (len(X.iloc[0])):
    if X.iloc[:,i].dtype == object:
        X.iloc[:,i] = lb_coder.fit_transform(X.iloc[:,i])
print(dataset.dtypes)


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


/home/egebasturk/anaconda3/envs/CS464/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
#Splitting the data into train/test sets
    #Importing the corresponding function
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [23]:
#Standard Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

/home/egebasturk/anaconda3/envs/CS464/lib/python2.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/egebasturk/anaconda3/envs/CS464/lib/python2.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Decision Tree


In [ ]:

#Decision Tree Classification
from sklearn import tree
dtc = tree.DecisionTreeClassifier(max_leaf_nodes=200) #max_leaf_nodes parametresi
dtc.fit(X_train, y_train)
#Printing Accuracy 
print('Accuracy of Decision Tree Classificatier on Training Set: {:.8f}'
     .format(dtc.score(X_train, y_train)))
print('Accuracy of Decision Tree Classificatier on Test Set: {:.8f}'
     .format(dtc.score(X_test, y_test)))


KNN


In [ ]:

#KNN
from sklearn.neighbors import KNeighborsClassifier
knn_c = KNeighborsClassifier(n_neighbors=3) # Parameter to be changed
knn_c.fit(X_train, y_train)
#Printing Accuracy 
print('Accuracy of KNN Classificatier on Training Set: {:.8f}'
     .format(knn_c.score(X_train, y_train)))
print('Accuracy of KNN Classificatier on Test Set: {:.8f}'
     .format(knn_c.score(X_test, y_test)))


Logistic Regression


In [ ]:

#Logistic Regression
from sklearn.linear_model import LogisticRegression
logistic_regressor = LogisticRegression()
logistic_regressor.fit(X_train, y_train)
#Printing Accuracy 
print('Accuracy of Logistic Regression on Training Set: {:.8f}'
     .format(logistic_regressor.score(X_train, y_train)))
print('Accuracy of Logistic Regression on Test Set: {:.8f}'
     .format(logistic_regressor.score(X_test, y_test)))


Neural Network


In [24]:
#Neural Net
from keras.models import Sequential
from keras.layers import Dense


In [25]:

model = Sequential()
model.add(Dense(20, input_dim=19, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# Model Compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Model Fitting
history = model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=0)


In [26]:
#Evaluation
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Predict
predictions = model.predict(X_test)
# Round predictions
predictions = [round(x[0]) for x in predictions]


NameError: name 'features_train' is not defined

In [22]:
# Saves prediction to a new csv for manual control
mat ={'Real': np.squeeze(np.asarray(X_test)), 'Predicted': np.squeeze(np.asarray(predictions))}
dataFrame = pa.DataFrame(data=mat
dataset.replace(('Yes', 'No', 'Male', 'Female', 'No phone service', 'DSL', 'Fiber optic', 'No internet service', 'Month-to-month', 'One year', 'Two year', 'Mailed check', 'Electronic check', 'Credit card (automatic)', 'Bank transfer (automatic)', ' ')
                 , (1,    0,     1,       0,               2,          1,       2,              2,                        0,            1,            2,          0,             1,                   2,             3,                                  0), inplace=True)
#print(raw_data)
dataset[['TotalCharges']] = dataset[['TotalCharges']].apply(pa.to_numeric)

raw_data = dataset.astype({'MonthlyCharges': np.float32, 'TotalCharges':np.float32})

print(dataset.dtypes))
dataFrame.to_csv("../Data/telco-customer-churn/Predictions.csv")


NameError: name 'features_train' is not defined

In [27]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
acc = accuracy_score(y_test, predictions, normalize=True, sample_weight=None)
print("Accuracy:" + str(acc * 100) + "%")


NameError: name 'features_train' is not defined

In [ ]:
#Plot
plt.subplot(1,2,1)
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.axhline(y=acc, color='r')
plt.ylim(0.4,1)


NameError: name 'features_train' is not defined

In [31]:
#Complex one
model2 = Sequential()
model2.add(Dense(200, input_dim=19, activation='relu'))
model2.add(Dense(150, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(25, activation='relu'))
model2.add(Dense(1,activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history2 = model2.fit(X_train, y_train, epochs=100, batch_size=5, verbose=0)
scores = model2.evaluate(X_test, y_test)
predictions2 = model2.predict(X_test)
predictions2 = [round(x[0]) for x in predictions2]
acc = accuracy_score(y_test, predictions2, normalize=True, sample_weight=None)

plt.subplot(1,2,2)
plt.plot(history2.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.axhline(y=acc, color='r')
plt.ylim(0.4,1)
plt.show()


Epoch 1/100
   5/4225 [..............................] - ETA: 2s - loss: 0.6358 - acc: 0.8000

 250/4225 [>.............................] - ETA: 0s - loss: 0.5054 - acc: 0.7680

 490/4225 [==>...........................] - ETA: 0s - loss: 0.5005 - acc: 0.7510

 730/4225 [====>.........................] - ETA: 0s - loss: 0.4982 - acc: 0.7466

 970/4225 [=====>........................] - ETA: 0s - loss: 0.4932 - acc: 0.7526

...

2860/4225 [===================>..........] - ETA: 0s - loss: 0.3448 - acc: 0.8297

2965/4225 [====================>.........] - ETA: 0s - loss: 0.3430 - acc: 0.8307

3090/4225 [====================>.........] - ETA: 0s - loss: 0.3408 - acc: 0.8314

3205/4225 [=====================>........] - ETA: 0s - loss: 0.3404 - acc: 0.8303

KeyboardInterrupt: 

Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10) #Parameters to be changed
rfc.fit(X_train,y_train)
#Printing Accuracy 
print('Accuracy of RFC on Training Set: {:.8f}'
     .format(rfc.score(X_train, y_train)))
print('Accuracy of RFC on Test Set: {:.8f}'
     .format(rfc.score(X_test, y_test)))


NameError: name 'features_train' is not defined

In [81]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(X_train,y_train)
print('Accuracy of SVC on Training Set: {:.8f}'
     .format(svc.score(X_train, y_train)))
print('Accuracy of SVC on Test Set: {:.8f}'
     .format(svc.score(X_test, y_test)))

C:\Users\Bugra\Anaconda\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of SVC on Training Set: 0.80209443
Accuracy of SVC on Test Set: 0.79205110
